In [27]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
from pygam import LinearGAM, s
import statsmodels.api as sm
from scipy.stats import mannwhitneyu

In [28]:
# Here we load the data regarding the GLOBOCAN2022 cancer incidence and WHO HPV vaccine coverage
# Then perform a simple cleaning for the vaccine data, where we select about 10 years (+-2) before our cancer data, to account for HPV related cancer development
# We also select for the vaccination group of interest, in this case "HPV Vaccination program coverage, first dose, females"
# Now the mean vaccine coverage is computed for the 2010-2014 period
# Finally we merge the data in one dataset containing: WHO code, mean coverage 2010-2014, and ASR (Age Standardized Rate)
# The last step consist of setting the vaccine coverage missing values to zero, this is done as if there is no data, the vaccine has not been introduced yet

# Define WHO countries by region
who_countries = {
    "AFR": ["Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cameroon", "Cape Verde", "Cabo Verde", "Central African Republic", "Chad", "Comoros", "Ivory Coast", "Democratic Republic of the Congo", "Equatorial Guinea", "Eritrea", "Ethiopia", "Gabon", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Kenya", "Lesotho", "Liberia", "Madagascar", "Malawi", "Mali", "Mauritania", "Mauritius", "Mozambique", "Namibia", "Niger", "Nigeria", "Republic of the Congo", "Rwanda", "São Tomé and Príncipe", "Senegal", "Seychelles", "Sierra Leone", "South Africa", "South Sudan", "Eswatini", "Togo", "Uganda", "Tanzania", "Zambia", "Zimbabwe"],
    "AMR": ["Peru", "Paraguay", "Saint Kitts and Nevis", "Antigua and Barbuda", "Argentina", "Bahamas", "Barbados", "Belize", "Bolivia", "Brazil", "Canada", "Chile", "Colombia", "Costa Rica", "Cuba", "Dominica", "Dominican Republic", "Ecuador", "El Salvador", "Grenada", "Guatemala", "Guyana", "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua", "Panama", "Saint Lucia", "Saint Vincent and the Grenadines", "Suriname", "Trinidad and Tobago", "the United States of America", "Uruguay", "Venezuela"],
    "SEAR": ["Bangladesh", "Bhutan", "Democratic People's Republic of Korea", "India", "Maldives", "Myanmar", "Nepal", "Sri Lanka", "Thailand", "Timor-Leste"],
    "EUR": ["Albania", "Andorra", "Armenia", "Austria", "Azerbaijan", "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Georgia", "Germany", "Greece", "Hungary", "Iceland", "Ireland", "Israel", "Italy", "Kazakhstan", "Kyrgyzstan", "Latvia", "Lithuania", "Luxembourg", "Malta", "Moldova", "Monaco", "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland", "Portugal", "Romania", "Russia", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland", "Tajikistan", "Turkey", "Turkmenistan", "Ukraine", "United Kingdom", "Uzbekistan"],
    "EMR": ["Libya", "Afghanistan", "Bahrain", "Djibouti", "Egypt", "Iran", "Iraq", "Jordan", "Kuwait", "Israel", "Oman", "Pakistan", "Qatar", "Saudi Arabia", "Somalia", "Sudan", "Syria", "Tunisia", "United Arab Emirates", "Yemen", "Morocco"],
    "WPR": ["Australia", "Brunei", "Cambodia", "China", "Cook Islands", "Fiji", "Indonesia", "Japan", "Kiribati", "Laos", "Malaysia", "Marshall Islands", "Micronesia", "Mongolia", "Nauru", "New Zealand", "Niue", "Palau", "Papua New Guinea", "Philippines", "Samoa", "Singapore", "Solomon Islands", "South Korea", "Taiwan", "Tonga", "Tuvalu", "Vanuatu", "Vietnam"]
}

# Some countries have small variations in name, so a second dictionary accounts for that
name_recode = {
    "Bolivia (Plurinational State of)": "Bolivia",
    "Brunei Darussalam": "Brunei",
    "Côte d'Ivoire": "Ivory Coast",
    "Czechia": "Czech Republic",
    "Lao People's Democratic Republic": "Laos",
    "Republic of Moldova": "Moldova",
    "Russian Federation": "Russia",
    "Sao Tome and Principe": "São Tomé and Príncipe",
    "Türkiye": "Turkey",
    "United States of America": "the United States of America",
    "Micronesia (Federated States of)": "Micronesia",
    "Netherlands (Kingdom of the)": "Netherlands",
    "Republic of Korea": "South Korea",
    "Saint Kitts and Nevis": "Saint Kitts and Nevis",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "United Republic of Tanzania": "Tanzania"
}

# Read the vaccination data
vax = pd.read_csv("C:\\Users\\ettod\\Tubingen_exercises\\DataL\\cacertovacc\\who_vax_country.tsv", sep="\t", header=0)

# Subset for years 2010-2014
vax = vax[vax['YEAR'].isin(range(2010, 2014))]

# Subset for specific antigen description
vax = vax[vax['ANTIGEN_DESCRIPTION'] == "HPV Vaccination program coverage, first dose, females"]

# Convert COVERAGE to integer
vax['COVERAGE'] = vax['COVERAGE'].astype('Int64')

# Group by country and calculate mean coverage
vaccine_summary = vax.groupby('NAME').agg(mean_coverage_2010_2014=('COVERAGE', 'mean')).reset_index()

# Read the cancer data
country_canc = pd.read_csv("C:\\Users\\ettod\\Tubingen_exercises\\DataL\\cacertovacc\\Datlit_HPV_Project_Final_Database - females-2022-cervix-uteri.tsv", sep="\t", header=0)

# Convert ASR to numeric
country_canc['ASR (World)'] = pd.to_numeric(country_canc['ASR (World)'])

# Select columns
canc_country = country_canc[['Label', 'ASR (World)']].copy()
canc_country.columns = ['NAME', 'ASR']

# Merge vaccine summary with cancer data
region_summary = pd.merge(vaccine_summary, canc_country, on='NAME', how='inner')

# homogenize names
region_summary['NAME'] = region_summary['NAME'].replace(name_recode)

# Create WHO lookup DataFrame
who_lookup = pd.DataFrame([(country, region) for region, countries in who_countries.items() for country in countries], columns=['NAME', 'WHO_REGION'])

# Merge with region summary
region_summary = pd.merge(region_summary, who_lookup, on='NAME', how='left', validate= 'many_to_many')

# Replace NA with 0
region_summary['mean_coverage_2010_2014'] = region_summary['mean_coverage_2010_2014'].fillna(0)

# Convert COVERAGE to integer
region_summary['mean_coverage_2010_2014'] = region_summary['mean_coverage_2010_2014'].astype('Int64')

vax_1 = region_summary[region_summary["mean_coverage_2010_2014"] != 0]
vax_0 = region_summary[region_summary["mean_coverage_2010_2014"] == 0]

In [33]:
#
u, p = mannwhitneyu(vax_0["ASR"], vax_1["ASR"], alternative='two-sided')
print(f"Mann–Whitney U p-value: {p:.3f}")

exp = (len(vax_1)*len(vax_0))/2
print("expected U: ", exp, "\n Mann–Whitney U: ", u)

Mann–Whitney U p-value: 0.003
expected U:  1638.0 
 Mann–Whitney U:  2195.5


Do countries that had introduced HPV vaccination by 2010–2014 have different 2022 cancer incidence?

If I randomly pick one country from group A and one from group B, how often does one tend to have a higher value than the other?

There is strong evidence that cervical cancer incidence in 2022 differs between countries that had introduced HPV vaccination by 2010–2014 and those that had not (p = 0.003).

In [30]:
lowess_data = vax_1[['mean_coverage_2010_2014', 'ASR']]

lowess = sm.nonparametric.lowess(
    lowess_data['ASR'],
    lowess_data['mean_coverage_2010_2014'],
    frac=0.5
)


In [31]:
# Define colors for WHO regions
who_colors = {
    "AFR": "#1b9e77",
    "AMR": "#d95f02",
    "SEAR": "#7570b3",
    "EUR": "#e7298a",
    "EMR": "#66a61e",
    "WPR": "#e6ab02"
}

# Initialize the interactive scatter plot
fig = px.scatter(
    region_summary,
    x='mean_coverage_2010_2014',
    y='ASR',
    color='WHO_REGION',
    color_discrete_map=who_colors,
    hover_data={'NAME': True, 'mean_coverage_2010_2014': ':.2f', 'ASR': ':.2f', 'WHO_REGION': False},
    title='HPV Vaccination Coverage vs. Cervical Cancer Rate by WHO Region',
    labels={
        'mean_coverage_2010_2014': 'Mean HPV Vaccination Coverage (2010–2014)',
        'ASR': 'Age-Standardized Cervical Cancer Rate'
    }
)

# Extract smoothed x and y
x_smooth = lowess[:, 0]
y_smooth = lowess[:, 1]

# Add the LOWESS trend line
fig.add_trace(
    go.Scatter(
        x=x_smooth,
        y=y_smooth,
        mode='lines',
        name='Spearman Trend Line (LOWESS)',
        line=dict(color='black', width=2)
    )
)

# Customize layout
fig.update_layout(
    legend_title_text='WHO Region',
    hovermode='closest'
)
# Add y-axis range limit
fig.update_yaxes(range=[0, 40])
fig.update_xaxes(range=[0, 100])

# Show the plot
fig.show()